In [150]:
# load the diabete file and build a MLP model to predict the outcome of the diabete
import pandas as pd
data = pd.read_csv('diabetes.txt', sep=' ', header=None)
data.head()


,0,1,2,3,4,5,6,7,8,9
0,-1,1:-0.294118,2:0.487437,3:0.180328,4:-0.292929,5:-1,6:0.00149028,7:-0.53117,8:-0.0333333,NaN
1,1,1:-0.882353,2:-0.145729,3:0.0819672,4:-0.414141,5:-1,6:-0.207153,7:-0.766866,8:-0.666667,NaN
2,-1,1:-0.0588235,2:0.839196,3:0.0491803,4:-1,5:-1,6:-0.305514,7:-0.492741,8:-0.633333,NaN
3,1,1:-0.882353,2:-0.105528,3:0.0819672,4:-0.535354,5:-0.777778,6:-0.162444,7:-0.923997,8:-1,NaN
4,-1,1:-1,2:0.376884,3:-0.344262,4:-0.292929,5:-0.602837,6:0.28465,7:0.887276,8:-0.6,NaN


In [151]:
# from column 1 to 8 are the features, 0 are labels, splits the columns from 1 to 8 and keep second value
X = data.iloc[:, 1:9]
# split the columns from 0 to 7 and keep second value, convert to float
X = X.apply(lambda x: x.str.split(':').str[1].astype(float))
X

,1,2,3,4,5,6,7,8
0,-0.294118,0.487437,0.180328,-0.292929,-1.000000,0.001490,-0.531170,-0.033333
1,-0.882353,-0.145729,0.081967,-0.414141,-1.000000,-0.207153,-0.766866,-0.666667
2,-0.058824,0.839196,0.049180,-1.000000,-1.000000,-0.305514,-0.492741,-0.633333
3,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,-1.000000
4,-1.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000
...,...,...,...,...,...,...,...,...
763,0.176471,0.015075,0.245902,-0.030303,-0.574468,-0.019374,-0.920581,0.400000
764,-0.764706,0.226131,0.147541,-0.454545,-1.000000,0.096870,-0.776260,-0.800000
765,-0.411765,0.216080,0.180328,-0.535354,-0.735225,-0.219076,-0.857387,-0.700000
766,-0.882353,0.266332,-0.016393,-1.000000,-1.000000,-0.102832,-0.768574,-0.133333


In [152]:
# fill the missing values with the mean of the column
import numpy as np
X = X.fillna(X.mean())
Y = data.iloc[:, 0].astype(float)
Y = (Y + 1)//2
Y
X = np.array(X)
Y = np.array(Y)


In [158]:
# Create a MLP model using Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# the output is -1 or 1, so the activation function is sigmoid
# build a MLP model
# 
def create_model():

    model = keras.Sequential([
        layers.Dense(20, activation='relu', input_shape=[8]),
        layers.Dense(20, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(1, activation='sigmoid'),
    ])
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model


In [134]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [135]:
# train,dev,test split
from sklearn.model_selection import train_test_split
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.8, random_state=42)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_temp, Y_temp, train_size=0.5, random_state=42)

In [136]:
# train with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=10,
    verbose=0,
    mode='auto',
    restore_best_weights=True
)
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_dev, Y_dev), callbacks=[early_stopping])
model.evaluate(X_test, Y_test)
# model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_dev, Y_dev))

Epoch 1/100


20/20 [==============================] - 1s 24ms/step - loss: 0.6806 - accuracy: 0.6091 - val_loss: 0.6551 - val_accuracy: 0.7532
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6474 - accuracy: 0.7150 - val_loss: 0.6256 - val_accuracy: 0.7143
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6203 - accuracy: 0.7166 - val_loss: 0.5962 - val_accuracy: 0.7273
Epoch 4/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5930 - accuracy: 0.7166 - val_loss: 0.5721 - val_accuracy: 0.7013
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5680 - accuracy: 0.7231 - val_loss: 0.5505 - val_accuracy: 0.7403
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5443 - accuracy: 0.7362 - val_loss: 0.5355 - val_accuracy: 0.7403
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5264 - accuracy: 0.7459 - val_loss: 0.5232 - val_accuracy: 0.7532
Epoch 8/100
20/20 [=====

[0.4646388590335846, 0.7662337422370911]

In [160]:
seed = 7
np.random.seed(seed)
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, Y):
    model = create_model()
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0, callbacks=[early_stopping])
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print(f"{model.metrics_names[1]}: {scores[1]*100:.2f}%")
    cvscores.append(scores[1] * 100)

print(f"{np.mean(cvscores):.2f}% (+/- {np.std(cvscores):.2f}%)")


KeyboardInterrupt: 